In [1]:
%use kotlin-statistics, krangl, kravis, klaxon

In [2]:
import kotlin.math.*

In [3]:
@file:Repository("https://dl.bintray.com/kyonifer/maven")
@file:DependsOn("com.kyonifer:koma-core-ejml:0.12")
@file:DependsOn("com.kyonifer:koma-plotting:0.12")

In [4]:
// @file:Repository("https://repo1.maven.org/maven2")
@file:DependsOn("com.github.haifengl:smile-core:2.2.2")

In [5]:
import koma.extensions.*
import koma.*

In [15]:
import smile.math.distance.DynamicTimeWarping

In [25]:
import smile.clustering.HierarchicalClustering
import smile.clustering.linkage.CompleteLinkage

In [8]:
class Action(val time: Long, val group_id: String, val event_id: String, val dt: String) {
    fun print() {
        println("$time $group_id $event_id $dt")
    }

    fun getEvent(): String {
        return "$group_id; $event_id; $dt"
    }
}


In [10]:
class Session(val id: String, val startTime: Long = 0) {
    var actions: ArrayList<Action> = ArrayList()

    fun add(time: Long, group: String, event: String, action: String) {
        actions.add(Action(time,group,event, action))
    }

    fun print() {
        println(id)
        for (action in actions) {
            action.print()
        }
    }

    fun time(): Long {
        return actions[actions.size - 1].time
    }

    fun events(): String {
        val events = ArrayList<String>()
        for (action in actions)
            events.add(action.getEvent())
        return events.joinToString(separator = " , ")
    }



    fun times(): String {
        val times = ArrayList<String>()
        for (action in actions)
            times.add(action.time.toString())
        return times.joinToString(separator = " , ")
    }
}


In [11]:
fun sessionsFromTSV(path: String): ArrayList<Session> {
    val df = DataFrame.readTSV(path)
    val ses = ArrayList<Session>()
    for (i in 0 until df.nrow) {
        val events = df["events"][i].toString().split(" , ")
//        if (events.size < 3)
//            continue
        val times = df["timestamps"][i].toString().split(" , ")
        ses.add(Session(df["session_id"][i].toString()))
        for (j in events.indices) {
            val params = events[j].split(" ;")
            ses[ses.size - 1].add(times[j].toLong(), params[0], params[1], params[2])
        }
    }
    return ses
}

In [12]:
class ActionDist : smile.math.distance.Distance<Action> {
    override fun d(x: Action, y: Action): Double {
        var score = 1.0
        if (x.group_id != y.group_id)
            return score
        score -= 1.0 / 2
        if (x.event_id != y.event_id)
            return score
        score -= 1.0 / 4

        if (x.dt != y.dt)
            return score
        return 0.0
    }

}

In [20]:
val dtw = DynamicTimeWarping(ActionDist())

fun DTWSession(s1: Session, s2: Session): Double {
    val a1 = Array(s1.actions.size) { i -> s1.actions[i]}
    val a2 = Array(s2.actions.size) { i -> s2.actions[i]}
    return dtw.apply(a1, a2) / (a1.size + a2.size)
}

In [31]:
class DWTDist<T> : smile.math.distance.Distance<T> {
    
    public fun DWTDist(a: Int) {
        print(a)
    }
    
    override fun d(x: T, y: T): Double {
        if (x is Session && y is Session)
            return DTWSession(x, y)
        return 1.0
    }

}

In [32]:
val ses = sessionsFromTSV("../data/UI_1-2buckets_7march_session.tsv")

In [34]:
val t = DWTDist<Session> ()

java.lang.NoSuchMethodError: Line_34_jupyter$DWTDist: method <init>()V not found
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:95)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:40)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
kotlin.script.experimental.jvmhost.repl.JvmReplEvaluator$eval$$inlined$write$lambda$2.invokeSuspend(legacyReplEvaluation.kt:57)
kotlin.coroutines.jvm.internal.BaseContinuationImpl.resumeWith(ContinuationImp

In [26]:
val clusters = HierarchicalClustering.fit(CompleteLinkage.of(ses.toArray(), DWTDist()))

java.lang.NoSuchMethodError: Line_20_jupyter$DWTDist: method <init>()V not found
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:95)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:40)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
kotlin.script.experimental.jvmhost.repl.JvmReplEvaluator$eval$$inlined$write$lambda$2.invokeSuspend(legacyReplEvaluation.kt:57)
kotlin.coroutines.jvm.internal.BaseContinuationImpl.resumeWith(ContinuationImp